In [1]:
import ibm_db
import ibm_db_dbi
import pandas

In [2]:
dsn_hostname = "dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net"
dsn_uid = "mxm64951"
dsn_pwd = "2fmb5ql-vz62q9dv"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"
dsn_port = "50000"
dsn_protocol = "TCPIP"

In [3]:
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=mxm64951;PWD=2fmb5ql-vz62q9dv;


In [4]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)
    
except:
    print("Unable to connect: ", ibm_db.conn_errormsg())

Connected to database:  BLUDB as user:  mxm64951 on host:  dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net


In [5]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

DBMS_NAME:  DB2/LINUXX8664
DBMS_VER:   11.01.0404
DB_NAME:    BLUDB


In [6]:
client = ibm_db.client_info(conn)

print ("DRIVER_NAME:          ", client.DRIVER_NAME) 
print ("DRIVER_VER:           ", client.DRIVER_VER)
print ("DATA_SOURCE_NAME:     ", client.DATA_SOURCE_NAME)
print ("DRIVER_ODBC_VER:      ", client.DRIVER_ODBC_VER)
print ("ODBC_VER:             ", client.ODBC_VER)
print ("ODBC_SQL_CONFORMANCE: ", client.ODBC_SQL_CONFORMANCE)
print ("APPL_CODEPAGE:        ", client.APPL_CODEPAGE)
print ("CONN_CODEPAGE:        ", client.CONN_CODEPAGE)

DRIVER_NAME:           libdb2.a
DRIVER_VER:            11.01.0404
DATA_SOURCE_NAME:      BLUDB
DRIVER_ODBC_VER:       03.51
ODBC_VER:              03.01.0000
ODBC_SQL_CONFORMANCE:  EXTENDED
APPL_CODEPAGE:         1208
CONN_CODEPAGE:         1208


In [7]:
# Drop the table INSTRUCTOR in case it exists
dropQuery = "drop table INSTRUCTOR1"

# Execute the drop statement
dropStmt = ibm_db.exec_immediate(conn, dropQuery)

In [8]:
# Construct the Create Table DDL statement
createQuery = "create table INSTRUCTOR1(id INTEGER PRIMARY KEY NOT NULL, fname VARCHAR(20), \
lname VARCHAR(20), city VARCHAR(20), ccode CHAR(4))"

# Execute the statement
createStmt = ibm_db.exec_immediate(conn, createQuery)

In [9]:
insertQuery = "INSERT INTO INSTRUCTOR1(id, fname, lname, city, ccode) \
VALUES (1, 'Rav', 'Ahuja', 'TORONTO', 'CA')"

#execute the insert statement
insertStmt = ibm_db.exec_immediate(conn, insertQuery)

In [10]:
insertQuery2 = "INSERT INTO INSTRUCTOR1 VALUES \
(2, 'Raul', 'Chong', 'Markham', 'CA'), (3, 'Hima', 'Vasudevan', 'Chicago', 'US')"

#execute the statement
insertStmt2 = ibm_db.exec_immediate(conn, insertQuery2)

In [11]:
selectQuery = "select * from INSTRUCTOR1"

selectStmt = ibm_db.exec_immediate(conn, selectQuery)

ibm_db.fetch_both(selectStmt)

{'ID': 1,
 0: 1,
 'FNAME': 'Rav',
 1: 'Rav',
 'LNAME': 'Ahuja',
 2: 'Ahuja',
 'CITY': 'TORONTO',
 3: 'TORONTO',
 'CCODE': 'CA  ',
 4: 'CA  '}

In [12]:
while ibm_db.fetch_row(selectStmt) != False:
    print(" ID:",  ibm_db.result(selectStmt, 0), " FNAME:",  ibm_db.result(selectStmt, "FNAME"))

 ID: 2  FNAME: Raul
 ID: 3  FNAME: Hima


In [13]:
update_query = "UPDATE INSTRUCTOR1 SET CITY='MOOSETOWN' WHERE fname='Rav'"

update_stmt = ibm_db.exec_immediate(conn, update_query)

In [14]:
# Connection for pandas
pconn = ibm_db_dbi.Connection(conn)

In [15]:
# Retrieve all rows in INSTRUCTOR table
selectQuery = "select * from INSTRUCTOR1"

# Retrieve the query results into a pandas dataframe
df = pandas.read_sql(selectQuery, pconn)

# Print just the LNAME for first row in the pandas data frame
df.LNAME[1]

'Chong'

In [16]:
df

,ID,FNAME,LNAME,CITY,CCODE
0,1,Rav,Ahuja,MOOSETOWN,CA
1,2,Raul,Chong,Markham,CA
2,3,Hima,Vasudevan,Chicago,US


In [17]:
ibm_db.close(conn)

True